# Overview

If the student asks questions in Vietnamese, please add the standard answer: “We're sorry for any inconvenience; however, StarLeo can only answer questions in English. Unfortunately, Vietnamese isn't available at the moment. Thank you for your understanding!”

# Set up

In [4]:
import os
import pandas as pd
pd.set_option("display.max_colwidth", None)

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dalle3-swo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e51119f8d8774069a6594d92ccf7a70d"

In [5]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Optional
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

gpt_4o = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4o",
    temperature=0
)
gpt_35_turbo_16k = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
) 


In [8]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = """
Given the user question below, detect the language of it as either `Vietnamese`, or `Other`.

Do not respond with more than one word.

<question>
{input}
</question>

Language:"""

language_detection_chain = (
    PromptTemplate.from_template(prompt_template)
    | gpt_4o
    | StrOutputParser()
)

In [9]:
language_detection_chain.invoke({"input": "I need to attend my grandfather''s 49th memorial day? Am I allowed to be absent from class?"})

'Other'

In [21]:
sio_chain = (
    PromptTemplate.from_template("{input}")
    | llm
    | StrOutputParser()
)

In [22]:
def route(info):
    if info["language"] == "Vietnamese":
        return """We're sorry for any inconvenience; however, StarLeo can only answer questions in English. Unfortunately, Vietnamese isn't available at the moment. Thank you for your understanding!"""
    else:
        return sio_chain

In [27]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

full_chain = RunnablePassthrough.assign(language=language_detection_chain) | RunnableLambda(route)

In [29]:
full_chain.invoke({"input": "nếu tôi thi không pass thì sao?"})

"We're sorry for any inconvenience; however, StarLeo can only answer questions in English. Unfortunately, Vietnamese isn't available at the moment. Thank you for your understanding!"